In [17]:
import os
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

SyntaxError: invalid syntax (<ipython-input-17-4bf837d1dfd4>, line 2)

- 이미지 이름과 이미지의 경로, csv파일의 경로를 불러온다.

In [7]:
img_name = ['normal', 'mask1', 'mask2', 
            'mask3', 'mask4', 'mask5', 'incorrect_mask']

csv_path = '/opt/ml/input/data/train/train.csv'
data_path = '/opt/ml/input/data/train/images'
mask_image_frame = pd.read_csv(csv_path)

In [8]:
mask_image_frame.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


- 데이터셋의 정의

In [9]:
class MaskImageDataset(Dataset):
    
    def __init__(self, csv_file, data_path, transform=None):
        """
        Args:
            csv_file (string): csv_file 경로
            data_path (string): data_path 경로
            transform (string): 샘플에 적용될 transform(전처리)
        """
        self.mask_image_frame = pd.read_csv(csv_file)
        self.data_path = data_path
        self.transform = transform
    
    
    def __len__(self):
        return len(self.mask_image_frame) * 7
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        idx //= 7
        img_path = os.path.join(self.data_path, 
                                self.mask_image_frame.loc[idx,'path'])
        
        img_type_list = ['.png', '.jpg', '.jpeg']
        for img_type in img_type_list:
            if os.path.isfile(os.path.join(img_path, img_name[idx%7] + img_type)):
                image = Image.open(os.path.join(img_path, img_name[idx%7] + img_type))
                break
        
        if self.transform:
            image = self.transform(image)
        
        label = { 'status': idx%7,
                  'gender': self.mask_image_frame.loc[idx,'gender'],
                  'age': self.mask_image_frame.loc[idx,'age']
                 }
            
        return image, label

- torchvision의 transforms 기능으로 전처리 및 augmentation, 그리고 data_loader

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
    ])

dataset = MaskImageDataset(csv_path, data_path, data_transform)
data_loader = DataLoader(dataset,
                            shuffle=True,
                            batch_size=10, 
                            num_workers=8,)

for data in tqdm.notebook.tqdm(data_loader):
    pass

  0%|          | 0/1890 [00:00<?, ?it/s]